## **Import Library**

In [3]:
import os
import shutil
from sklearn.model_selection import train_test_split
from google.colab import drive
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
import numpy as np

# **Mount Google Drive**

In [4]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Functions for Creating a Directory**

In [3]:
# Function to create a directory if it does not exist
def create_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

# **Path ke Dataset**

In [4]:
# Path to dataset
dataset_path = '/content/drive/My Drive/Historia/dataset'

# Verify if the dataset path is correct
if not os.path.exists(dataset_path):
    raise FileNotFoundError(f"Path {dataset_path} not found. Make sure the path is correct and Google Drive is mounted correctly.")


# **Path to Train, Validation, and Test Directory**

In [5]:
# Path to train, validation, and test directories
train_path = '/content/data/train'
validation_path = '/content/data/validation'
test_path = '/content/data/test'

# Percentage of data for each set
train_size = 0.8
validation_size = 0.1
test_size = 0.1

# **Create Train, Validation, and Test Directories**

In [6]:
# Create train, validation, and test directories
create_dir(train_path)
create_dir(validation_path)
create_dir(test_path)

# **Valid Image File Extensions**

In [7]:
# Valid image file extensions
valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.gif')

# **Loop Through Every Folder in the Dataset**

In [8]:
# Loop through each folder (place) in the dataset
for place in os.listdir(dataset_path):
    place_path = os.path.join(dataset_path, place)

    if os.path.isdir(place_path):
        # Create subdirectories for each place in train, validation, and test
        create_dir(os.path.join(train_path, place))
        create_dir(os.path.join(validation_path, place))
        create_dir(os.path.join(test_path, place))

        # Get a list of all image files in the folder where
        images = [img for img in os.listdir(place_path) if img.lower().endswith(valid_extensions)]

        # Check if any images were found
        if not images:
            print(f"No images found in {place_path}. Skipping...")
            continue  # Skip to the next folder if no images are found

        # split data train, validation, dan test
        train_images, temp_images = train_test_split(images, test_size=(validation_size + test_size))
        validation_images, test_images = train_test_split(temp_images, test_size=(test_size / (validation_size + test_size)))

        # copy image to train directory
        for image in train_images:
            src = os.path.join(place_path, image)
            dst = os.path.join(train_path, place, image)
            try:
                shutil.copyfile(src, dst)
            except Exception as e:
                print(f"Error copying {src} to {dst}: {e}")

        # copy image to validation directory
        for image in validation_images:
            src = os.path.join(place_path, image)
            dst = os.path.join(validation_path, place, image)
            try:
                shutil.copyfile(src, dst)
            except Exception as e:
                print(f"Error copying {src} to {dst}: {e}")

        # copy image to test directory
        for image in test_images:
            src = os.path.join(place_path, image)
            dst = os.path.join(test_path, place, image)
            try:
                shutil.copyfile(src, dst)
            except Exception as e:
                print(f"Error copying {src} to {dst}: {e}")
print("Data berhasil dipisahkan ke train, validation, dan test set.")

Data berhasil dipisahkan ke train, validation, dan test set.


# **Directory for Datasets and Hyperparameters**

In [9]:
# Directory for datasets
train_dir = '/content/drive/MyDrive/Historia/data/train'
validation_dir = '/content/drive/MyDrive/Historia/data/validation'
test_dir = '/content/drive/MyDrive/Historia/data/test'

# Hyperparameters
learning_rate = 0.0001
dropout_rate = 0.5
batch_size = 32
l2_regularization = 0.01


# **Image Augmentation**

In [10]:
# More extensive image augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# **Load Dataset**

In [11]:
# Load dataset
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 348 images belonging to 5 classes.
Found 44 images belonging to 5 classes.
Found 45 images belonging to 5 classes.


# **Optimizer**

In [12]:
# Optimizer
optimizer = Adam(learning_rate=learning_rate)

# **Architecture Model**

In [13]:
# Model
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(512, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(512, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(1024, activation='relu', kernel_regularizer=l2(l2_regularization)),
    Dropout(dropout_rate),
    Dense(512, activation='relu', kernel_regularizer=l2(l2_regularization)),
    Dropout(dropout_rate),
    Dense(len(train_generator.class_indices), activation='softmax')
])

# **Callbacks**

In [14]:
# Callback to reduce learning rate when validation loss is not reduced
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-7, verbose=1)

# Callback to stop training early if validation loss is not reduced
class MyEarlyStopping(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        val_acc = logs.get('val_accuracy')
        if val_acc >= 0.9:
            self.model.stop_training = True
            print("Reached 90% validation accuracy! Stopping training.")

early_stopping = MyEarlyStopping()

# **Compile Model**

In [15]:
# Compile model
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# **Calculate Steps Per Epoch**

In [16]:
# Count steps per epoch
steps_per_epoch = max(1, train_generator.samples // batch_size)
validation_steps = max(1, validation_generator.samples // batch_size)

# **Train Model**

In [17]:
# Train model
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    epochs=150,
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/150
10/10 [==============================] - 246s 23s/step - loss: 28.6529 - accuracy: 0.3196 - val_loss: 27.2358 - val_accuracy: 0.2500 - lr: 1.0000e-04
Epoch 2/150
10/10 [==============================] - 210s 21s/step - loss: 27.8639 - accuracy: 0.4873 - val_loss: 27.1792 - val_accuracy: 0.2188 - lr: 1.0000e-04
Epoch 3/150
10/10 [==============================] - 202s 20s/step - loss: 27.5571 - accuracy: 0.4810 - val_loss: 27.2253 - val_accuracy: 0.1875 - lr: 1.0000e-04
Epoch 4/150
10/10 [==============================] - 203s 20s/step - loss: 27.2471 - accuracy: 0.5127 - val_loss: 27.1050 - val_accuracy: 0.1562 - lr: 1.0000e-04
Epoch 5/150
10/10 [==============================] - 213s 21s/step - loss: 26.9920 - accuracy: 0.5095 - val_loss: 27.4560 - val_accuracy: 0.2188 - lr: 1.0000e-04
Epoch 6/150
10/10 [==============================] - 200s 20s/step - loss: 26.7178 - accuracy: 0.5538 - val_loss: 27.5408 - val_accuracy: 0.1562 - lr: 1.0000e-04
Epoch 7/150
10/10 [=========

# **Model Evaluation**

In [17]:
# Model Evaluation
steps = max(1, test_generator.samples // test_generator.batch_size)
loss, accuracy = model.evaluate(test_generator, steps=steps)
print(f'Test accuracy: {accuracy * 100:.2f}%')

# **Save Model**

In [18]:
# Save Model
model.save('building_recognition_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# **Path to Image and Load Saved Model**

In [5]:
# Path to the downloaded image
image_path = '/content/drive/MyDrive/Historia/dataset/tugu yogyakarta/IMG_8947.JPG'

# Load the saved model
model = tf.keras.models.load_model('/content/drive/MyDrive/Historia/building_recognition_model.h5')


# **Image Preprocess Function**

In [6]:
# Preprocess gambar
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalisasi gambar
    return img_array

# **Image Preprocess and Prediction**

In [7]:
# Image preprocess and prediction
img_array = preprocess_image(image_path)
predictions = model.predict(img_array)[0]  # Get the first prediction (since batch size is 1)

1/1 [==============================] - 0s 420ms/step


# **Mapping Index Labels to Class Names**

In [8]:
# Recreate train_generator to get class_indices
train_datagen = ImageDataGenerator(rescale=1.0/255.0)
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Historia/data/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)
class_indices = train_generator.class_indices
labels = dict((v,k) for k,v in class_indices.items())


Found 348 images belonging to 5 classes.


# **Display the Percentage of Each Possible Class**

In [9]:
# Display the percentage of each possible class
for i, prob in enumerate(predictions):
    print(f'{labels[i]}: {prob * 100:.2f}%')

predicted_class = np.argmax(predictions)
predicted_label = labels[predicted_class]
print(f'Predicted class: {predicted_label}')


benteng vredeburg: 0.32%
candi prambanan: 6.21%
candi ratu boko: 7.07%
monumen jogja kembali: 4.33%
tugu yogyakarta: 82.08%
Predicted class: tugu yogyakarta
